In [24]:
#!pip uninstall protobuf
#!pip install protobuf==3.20.3

In [25]:
#pip install tensorflow

In [26]:
#!pip uninstall tensorflow
!pip install tensorflow==2.17.1
!pip install tf-models-official==2.17.0
#!pip uninstall protobuf
!pip install protobuf==3.20.3

  Using cached tf_models_official-2.17.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_model_optimization-0.8.0-py2.py3-none-any.whl.metadata (904 bytes)
  Using cached tensorflow_text-2.17.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [1]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import urllib
import cv2
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util, visualization_utils as vis_util
import os
import urllib.request
from google.protobuf import text_format
from object_detection.protos import string_int_label_map_pb2
import requests # Import requests library

# Download the pre-trained model (SSD MobileNet v2 trained on COCO)
#MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_FILE = tf.keras.utils.get_file(MODEL_NAME + '.tar.gz',
                                     'http://download.tensorflow.org/models/object_detection/{}.tar.gz'.format(MODEL_NAME),
                                     untar=True)

# Load the model into TensorFlow
# Search for the 'saved_model' directory within the extracted archive
PATH_TO_CKPT = MODEL_FILE + '/saved_model'
for root, dirs, files in os.walk(MODEL_FILE):
    if 'saved_model' in dirs:
        PATH_TO_CKPT = os.path.join(root, 'saved_model')
        break  # Stop searching once found

detection_model = tf.saved_model.load(PATH_TO_CKPT)

# Load the COCO labels
# Updated URL to a publicly accessible raw file on GitHub
PATH_TO_LABELS = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

# Download the label map file using requests instead of urllib.request
# This is because urllib might have issues with some firewalls or security settings
response = requests.get(PATH_TO_LABELS)
response.raise_for_status()  # Raise an exception for bad status codes (like 403)
label_map_string = response.text  # Get the content as text


# Parse the label map string
label_map = string_int_label_map_pb2.StringIntLabelMap()
text_format.Merge(label_map_string, label_map)

categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
category_index = {item['id']: item for item in categories}

# Replace this URL with your image URL
image_url = "https://raw.githubusercontent.com/geoffswc/UCSF-Research-AI-Symposium-2025/main/cartoon.png"
#image_url = "https://raw.githubusercontent.com/geoffswc/UCSF-Research-AI-Symposium-2025/main/metadata.png"
# Download and read the image
image_path = urllib.request.urlretrieve(image_url, 'image.png')[0]
image_np = cv2.imread(image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

# Convert image to tensor and run the detection
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]

# Perform detection
output_dict = detection_model.signatures['serving_default'](input_tensor)

# Extract useful information (boxes, classes, and scores)
boxes = output_dict['detection_boxes'][0].numpy()
classes = output_dict['detection_classes'][0].numpy().astype(np.int64)
scores = output_dict['detection_scores'][0].numpy()

print("printing")

# Print the detected labels and their confidence scores
for i in range(len(classes)):
  if scores[i] > 0.01:
    label = category_index[classes[i]]['name']
    print(f"Label: {label}, Confidence: {scores[i]:.2f}")

printing
Label: person, Confidence: 0.94
Label: person, Confidence: 0.87
Label: tv, Confidence: 0.47
Label: bottle, Confidence: 0.39
Label: tv, Confidence: 0.33
Label: clock, Confidence: 0.30
Label: book, Confidence: 0.20
Label: book, Confidence: 0.19
Label: person, Confidence: 0.14
Label: refrigerator, Confidence: 0.12
Label: clock, Confidence: 0.12
Label: person, Confidence: 0.10
Label: refrigerator, Confidence: 0.10
Label: tv, Confidence: 0.08
Label: tv, Confidence: 0.07
Label: laptop, Confidence: 0.06
Label: book, Confidence: 0.06
Label: tv, Confidence: 0.06
Label: book, Confidence: 0.06
Label: scissors, Confidence: 0.06
Label: clock, Confidence: 0.05
Label: scissors, Confidence: 0.04
Label: tv, Confidence: 0.04
Label: bottle, Confidence: 0.04
Label: train, Confidence: 0.03
Label: book, Confidence: 0.03
Label: book, Confidence: 0.03
Label: tv, Confidence: 0.03
Label: person, Confidence: 0.03
Label: bus, Confidence: 0.03
Label: book, Confidence: 0.03
Label: person, Confidence: 0.02


In [2]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import urllib
import cv2
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util, visualization_utils as vis_util
import os
import urllib.request
from google.protobuf import text_format
from object_detection.protos import string_int_label_map_pb2
import requests  # Import requests library

# Download the pre-trained model (Faster R-CNN Inception v2)
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_FILE = tf.keras.utils.get_file(MODEL_NAME + '.tar.gz',
                                     'http://download.tensorflow.org/models/object_detection/{}.tar.gz'.format(MODEL_NAME),
                                     untar=True)

# Load the model into TensorFlow
PATH_TO_CKPT = MODEL_FILE + '/saved_model'
for root, dirs, files in os.walk(MODEL_FILE):
    if 'saved_model' in dirs:
        PATH_TO_CKPT = os.path.join(root, 'saved_model')
        break  # Stop searching once found

detection_model = tf.saved_model.load(PATH_TO_CKPT)

# Load the COCO labels
PATH_TO_LABELS = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

# Download the label map file using requests
response = requests.get(PATH_TO_LABELS)
response.raise_for_status()  # Raise an exception for bad status codes (like 403)
label_map_string = response.text  # Get the content as text

# Parse the label map string
label_map = string_int_label_map_pb2.StringIntLabelMap()
text_format.Merge(label_map_string, label_map)

categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
category_index = {item['id']: item for item in categories}

# Replace this URL with your image URL
image_url = "https://raw.githubusercontent.com/geoffswc/UCSF-Research-AI-Symposium-2025/main/cartoon.png"
# Download and read the image
image_path = urllib.request.urlretrieve(image_url, 'image.png')[0]
image_np = cv2.imread(image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

# Convert image to tensor and run the detection
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]

# Perform detection
output_dict = detection_model.signatures['serving_default'](input_tensor)

# Extract useful information (boxes, classes, and scores)
boxes = output_dict['detection_boxes'][0].numpy()
classes = output_dict['detection_classes'][0].numpy().astype(np.int64)
scores = output_dict['detection_scores'][0].numpy()

# Filter and display the results visually
for i in range(len(classes)):
    if scores[i] > 0.25:
        # Get the label and bounding box coordinates
        label = category_index[classes[i]]['name']

        print(f"Label: {label}, Confidence: {scores[i]:.2f}")
        box = boxes[i]

        # Draw bounding box
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (xmin * image_np.shape[1], xmax * image_np.shape[1],
                                      ymin * image_np.shape[0], ymax * image_np.shape[0])

        # Draw the box and label on the image
        image_np = cv2.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (255, 0, 0), 2)
        image_np = cv2.putText(image_np, f"{label}: {scores[i]:.2f}", (int(left), int(top)-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

# Save the image with bounding boxes and labels
output_image_path = '/content/output_image.png'
cv2.imwrite(output_image_path, cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

# Provide a link to download the image
from google.colab import files
files.download(output_image_path)

print(f"Image with bounding boxes saved to {output_image_path} and ready for download.")


Label: person, Confidence: 0.94
Label: person, Confidence: 0.87
Label: tv, Confidence: 0.47
Label: bottle, Confidence: 0.39
Label: tv, Confidence: 0.33
Label: clock, Confidence: 0.30


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Image with bounding boxes saved to /content/output_image.png and ready for download.


In [ ]:
files.download(output_image_path)

In [ ]:
for item in category_index.values():
    print(f"{item['id']}. {item['name']}")